### Searching closest pub on google and scraping comments for further analysis

In [1]:
# Libraries
import os
import time
import random
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(f'Current working directory: {os.getcwd()}')




Current working directory: c:\Users\Roger\Desktop\WI22\applied_data_science\ADS_PROJECT\scraping-reviews-from-googlemaps-main


## Define Pub to search for as of closest pub analysis

In [2]:
# Read the pub name from the text file
with open('best_pub_name.txt', 'r') as f:
    pub_name = f.read().strip()

print(pub_name)

Old Gregory Pub, Zürich


In [4]:
# Set options for Chrome-Driver (regular mode)
opts = Options()
opts.add_argument("--window-size=1000,1200")
ua_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0"
]
opts.add_argument("user-agent=" + random.choice(ua_list))
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# Website to scrape
driver.get('https://www.google.ch/maps')

# Click on the "Accept" button for the cookies popup
try:
    accept_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//button[contains(@aria-label, 'Alle ablehnen')]"))
    )
    accept_button.click()
except:
    pass

# Search for location
search_box = driver.find_element(By.XPATH, "//input[@name='q']")
search_box.send_keys(pub_name)
search_box.send_keys(Keys.RETURN)

# Wait for search results to load
# WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "//h1[contains(text(),'Gregory')]")))

# Click on the location
# driver.find_element(By.XPATH, "//h1[contains(text(),'Gregory')]").click()



# Wait for the location page to load and click on "Rezensionen"
try:
    review_section = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Rezensionen')]")))
    review_section.click()
except:
    pass

# Scroll down to load more reviews dynamically
SCROLL_PAUSE_TIME = 6
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Extract all comments using the class name
reviews = driver.find_elements(By.CSS_SELECTOR, "span.wiI7pd")
for review in reviews:
    print("Comment: ", review.text)

# Extract rating using the xpath name
rating_element = driver.find_element(By.XPATH, "/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[2]/div/div[2]/div[1]")
rating = rating_element.text
print("Rating: ", rating)

driver.quit()

Comment:  Wenn man nichts bestellt, obwohl alle anderen von der Gruppe etwas bestellt haben, wird man dumm angemacht. Wenn man dann mal etwas bestellt, kann man dies fast nicht trinken, da es so schlecht schmeckt. Die Barkeeperin ist so unfreundlich und frech. Absolut nicht empfehlenswert.
Comment:  Alle Mitarbeiterinnen an der Bar sind so aggressiv und grob. Obwohl ich schon ein Bier bestellt hatte, musste ich nach kurzer Zeit noch etwas anderes bestellen. Ich wurde praktisch erzwungen, sonst wäre ich rausgeschmissen worden. Unglaublich. Nie wieder hier.
Comment:  Die schlechten Bewertungen bei Google konnten wir gar nicht verstehen! Die Bedienung war sehr nett und hilfsbereit und auch der DJ (Dietschi) war mega cool! Wir hatten einen tollen Karaoke Abend, Zehn Punkte-gerne wieder!
Comment:  Leider eher eine Ü40 Bar. Angestellte waren ausserdem enorm unfreundlich und aggressiv. 5 Minuten nach Ankunft in der Bar wurden wir aggressiv vom Personal angemacht, wir sollten uns endlich etwas